In [1]:
from we_recommend_a_grade_10.methods import *

In [2]:
#gamma [0-1] min_nn [2-max_nn] max_nn [min_nn-..]
#n [2-..], metric='euclidean', raw_features [T, F]
vars = ['gamma', 'min_nn', 'max_nn', 'n', 'raw_features']
counts = [1, 1, 1, 1, 1]
mins = [ 1, 3, 15, 24, False ]
maxs = [ 1, 3, 15, 24, False  ]
types = [float, int, int, int, bool]
def cancel_if(vals):
    return vals[1] >= vals[2]
values = [
    pd.Series(np.linspace(mins[i], maxs[i], counts[i])).astype(types[i]).values
    for i in range(len(vars))
]

In [3]:
suites = []
for v0 in values[0]:
    for v1 in values[1]:
        for v2 in values[2]:
            for v3 in values[3]:
                for v4 in values[4]:
                    vals = [v0, v1, v2, v3, v4]
                    if cancel_if(vals):
                        continue
                    suites.append({vars[i] : vals[i] for i in range(len(vars))})

In [4]:
tr, ts = get_train_test_split()
models = []
metrics = []
number = 1
for suite in suites:
    print('starting suite %d/%d...'%(number, len(suites)))
    model = train_CFVB(tr, **suite)
    print('training complete')
    models.append(model)
    preds = run_CFVB(ts, model)
    metrics.append(single_metric(preds.dropna()))
    print('evaluation complete')
    number += 1

starting suite 1/1...
training complete
evaluation complete


In [5]:
metrics

[1.9432223987830333]

In [6]:
sorted = pd.Series(metrics, name='MRSE').sort_values()
sorted

0    1.943222
Name: MRSE, dtype: float64

In [7]:
pd.Series(suites).loc[sorted.index]

0    {'gamma': 1.0, 'min_nn': 3, 'max_nn': 15, 'n':...
dtype: object

In [8]:
df = pd.DataFrame(sorted)
df['suite'] = pd.Series(suites).loc[sorted.index]
for var in vars:
    df[var] = df.apply(lambda x : x['suite'][var], axis=1)
df = df.drop(columns=['suite'])
df.reindex()

,MRSE,gamma,min_nn,max_nn,n,raw_features
0,1.943222,1.0,3,15,24,False


In [9]:
model = train_CFVB(tr, **suites[0])
preds = run_CFVB(ts, model)

In [10]:
#explanations_CF(ts, model[0])

In [11]:
#single_metric(preds)

In [12]:
#recommended = pd.DataFrame(model[0].predict_for_user('U1001', res_Profile.placeID)).dropna()

AttributeError: 'int' object has no attribute 'predict_for_user'

In [13]:
preds['explanation'] = preds.apply(lambda x : explanations_VB(x.user, model[1], model[2]), axis=1)
preds.dropna()

,user,item,rating,prediction,explanation
215,U1001,132830,3.0,2.958333,You were recommended Tortas Locas Hipocampo be...
216,U1001,132825,5.0,2.958333,You were recommended Tortas Locas Hipocampo be...
689,U1001,135040,3.0,3.500000,You were recommended Tortas Locas Hipocampo be...
1064,U1002,132825,4.0,2.791667,You were recommended El Rinc�n de San Francisc...
1066,U1002,132862,6.0,3.416667,You were recommended El Rinc�n de San Francisc...
...,...,...,...,...,...
882,U1137,135085,5.0,3.916667,You were recommended La Estrella de Dimas beca...
886,U1137,135058,4.0,4.375000,You were recommended La Estrella de Dimas beca...
892,U1137,135042,6.0,4.375000,You were recommended La Estrella de Dimas beca...
894,U1137,135049,6.0,4.375000,You were recommended La Estrella de Dimas beca...


In [15]:
preds.to_excel('explanations_VB.xls')

In [ ]:
model[2]

In [ ]:
from we_recommend_a_grade_10.vectorization import vector_cols
model[1].predict(model[2][vector_cols])

In [ ]:
vectors = model[2][model[2].user == 'U1001']
proba = model[1].predict_proba(vectors[vector_cols])
values = proba[:, 0] * 0
for i in model[1].classes_:
    values += proba[:, i] * i
vectors['prediction'] = values
item = vectors.sort_values('prediction').iloc[-1]

In [ ]:
list(item[vector_cols].sort_values()[-2:].index)

In [ ]:
res_res = res_Profile.copy()
res_res.index = res_Profile.placeID
res_res.loc[item['item']]['name']